In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import math
import statistics
import re
import json

In [2]:
business=pd.read_csv("yelp_business.csv")

In [3]:
business= business[business.state == "NV"]
business= business[business.is_open == 1]


In [4]:
def category_splitter(category_string):
    categories = category_string.split(";")   
    return categories
business['categories'] = business['categories'].map(lambda x: category_splitter(x))


In [5]:
def first_list(category_list):
    category_list = category_list[0]
    return category_list
business['categories'] = business['categories'].map(lambda x: first_list(x).lower())
business['categories'] = business['categories'].map(lambda x: re.sub(r'\'', '', x).lower())

business.drop(["business_id","neighborhood","address","review_count","is_open"], axis=1)


,name,city,state,postal_code,latitude,longitude,stars,categories
6,"""BDJ Realty""",Las Vegas,NV,89128,36.207430,-115.268460,4.0,real estate services
7,"""Soccer Zone""",Las Vegas,NV,89128,36.197484,-115.249660,1.5,shopping
9,"""Detailing Gone Mobile""",Henderson,NV,89014,36.055825,-115.046350,5.0,automotive
26,"""Alfredo's Jewelry""",Las Vegas,NV,89119,36.085051,-115.119421,4.5,shopping
37,"""Pampered Hair Passionate about Hair""",Henderson,NV,89074,36.019495,-115.094722,5.0,hair salons
41,"""Task Electric""",Las Vegas,NV,89113,36.056382,-115.269332,4.5,home services
48,"""Star Nursery""",Las Vegas,NV,89122,36.105196,-115.056880,3.5,nurseries & gardening
49,"""Rock of Ages""",Las Vegas,NV,89109,36.116965,-115.185696,4.0,arts & entertainment
52,"""Flight Deck Bar & Grill""",Las Vegas,NV,89119,36.066914,-115.170848,4.0,nightlife
53,"""Subway""",Las Vegas,NV,89119,36.064652,-115.118954,2.5,fast food


In [6]:
business.to_csv("nv_data.csv")

In [7]:
# for typeahead words
raw_unique_cats = list(np.unique(business.categories))

unique_cats = [re.sub(r'\'', '', i).lower() for i in raw_unique_cats]

print(unique_cats)

['3d printing', 'acai bowls', 'accessories', 'accountants', 'active life', 'acupuncture', 'addiction medicine', 'adult', 'adult education', 'adult entertainment', 'advertising', 'aerial fitness', 'aerial tours', 'african', 'air duct cleaning', 'airlines', 'airport lounges', 'airport shuttles', 'airport terminals', 'airports', 'allergists', 'amateur sports teams', 'american (new)', 'american (traditional)', 'amusement parks', 'anesthesiologists', 'animal physical therapy', 'animal shelters', 'antiques', 'apartments', 'appliances', 'appliances & repair', 'appraisal services', 'aquarium services', 'aquariums', 'arabian', 'arcades', 'argentine', 'art classes', 'art galleries', 'art restoration', 'art schools', 'art supplies', 'artificial turf', 'arts & crafts', 'arts & entertainment', 'asian fusion', 'assisted living facilities', 'atv rentals/tours', 'auction houses', 'audiologist', 'auto customization', 'auto detailing', 'auto glass services', 'auto insurance', 'auto loan providers', 'aut

# Create JSON of Map Grid of Boxes

### Filter city area out of all businesses

In [8]:
# Las Vegas Area
min_long = -115.383748
max_long = -115.015706
min_lat = 35.937648
max_lat = 36.328755

city_biz = business[(business.latitude < max_lat) &
                         (business.latitude > min_lat) &
                         (business.longitude < max_long) &
                         (business.longitude > min_long)]

### Compute lat and long increments

In [9]:
# Compute degrees lat per mile (N-S) and degrees long per mile (E-W)
ns_deg_per_mi = 1/69.01
ew_deg_per_mi = 1/(math.cos(
                            statistics.mean([min_lat, max_lat])*(math.pi/180)
                            ) * 69.1710411)

# Compute miles NS and EW for selected city area
miles_ns = (max_lat - min_lat)/ns_deg_per_mi
miles_ew = (max_long - min_long)/ew_deg_per_mi

delta_long = (max_long - min_long)/miles_ew
delta_lat = (max_lat - min_lat)/miles_ns

### Get counts of number of each category in a grid box

In [10]:
def get_category_counts(minimum_lat, minimum_long, d_lat, d_long):
    category_dict = defaultdict(int)
    for column, biz in business.iterrows():
        if ((biz['latitude'] > minimum_lat) and (biz['latitude'] < minimum_lat + d_lat)) and ((biz['longitude'] > minimum_long) and (biz['longitude'] < minimum_long + d_long)):
            category_dict[biz['categories']] += 1
    if not category_dict:
        category_dict['no business found in this area'] += 1
    return category_dict
        

### Create list of grid box dictionaries

In [11]:
boxes = []
 
i = min_lat
while i < max_lat:
    j = min_long
    while j < max_long:
        cat_counts = get_category_counts(i, j, delta_lat, delta_long)
        box = {'min_lat': i,
              'max_lat': i + delta_lat,
              'min_long': j,
              'max_long': j + delta_long,
              'category_counts': cat_counts}
        print(box)
        boxes.append(box)
        j += delta_long
    i += delta_lat
        

{'min_lat': 35.937648, 'max_lat': 35.95213865352848, 'min_long': -115.383748, 'max_long': -115.36584798512823, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.937648, 'max_lat': 35.95213865352848, 'min_long': -115.36584798512823, 'max_long': -115.34794797025646, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.937648, 'max_lat': 35.95213865352848, 'min_long': -115.34794797025646, 'max_long': -115.33004795538469, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.937648, 'max_lat': 35.95213865352848, 'min_long': -115.33004795538469, 'max_long': -115.31214794051292, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.937648, 'max_lat': 35.95213865352848, 'min_long': -115.31214794051292, 'max_long': -115.29424792564114, 'category_counts': defaultdict(<class 'int'>, {'no business found in 

{'min_lat': 35.95213865352848, 'max_lat': 35.966629307056955, 'min_long': -115.09734776205167, 'max_long': -115.0794477471799, 'category_counts': defaultdict(<class 'int'>, {'beauty & spas': 1, 'retirement homes': 1})}
{'min_lat': 35.95213865352848, 'max_lat': 35.966629307056955, 'min_long': -115.0794477471799, 'max_long': -115.06154773230813, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.95213865352848, 'max_lat': 35.966629307056955, 'min_long': -115.06154773230813, 'max_long': -115.04364771743636, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.95213865352848, 'max_lat': 35.966629307056955, 'min_long': -115.04364771743636, 'max_long': -115.02574770256459, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.95213865352848, 'max_lat': 35.966629307056955, 'min_long': -115.02574770256459, 'max_long': -115.00784768769282, 'category_co

{'min_lat': 35.98111996058543, 'max_lat': 35.99561061411391, 'min_long': -115.33004795538469, 'max_long': -115.31214794051292, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.98111996058543, 'max_lat': 35.99561061411391, 'min_long': -115.31214794051292, 'max_long': -115.29424792564114, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.98111996058543, 'max_lat': 35.99561061411391, 'min_long': -115.29424792564114, 'max_long': -115.27634791076937, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.98111996058543, 'max_lat': 35.99561061411391, 'min_long': -115.27634791076937, 'max_long': -115.2584478958976, 'category_counts': defaultdict(<class 'int'>, {'child care & day care': 1})}
{'min_lat': 35.98111996058543, 'max_lat': 35.99561061411391, 'min_long': -115.2584478958976, 'max_long': -115.24054788102583, 'category_counts': defaultdict(<

{'min_lat': 35.99561061411391, 'max_lat': 36.01010126764238, 'min_long': -115.20474785128229, 'max_long': -115.18684783641052, 'category_counts': defaultdict(<class 'int'>, {'airport shuttles': 1, 'food': 1, 'beer': 1, 'automotive': 1, 'parks': 1, 'skin care': 1, 'restaurants': 1})}
{'min_lat': 35.99561061411391, 'max_lat': 36.01010126764238, 'min_long': -115.18684783641052, 'max_long': -115.16894782153875, 'category_counts': defaultdict(<class 'int'>, {'pet groomers': 1, 'real estate': 2, 'gas stations': 1, 'convenience stores': 1, 'event planning & services': 1})}
{'min_lat': 35.99561061411391, 'max_lat': 36.01010126764238, 'min_long': -115.16894782153875, 'max_long': -115.15104780666698, 'category_counts': defaultdict(<class 'int'>, {'insurance': 1, 'automotive': 2, 'religious organizations': 1, 'plumbing': 1, 'home services': 3, 'food': 2, 'pizza': 1, 'home decor': 1, 'water heater installation/repair': 1, 'education': 1, 'blow dry/out services': 1, 'elementary schools': 1, 'shoppi

{'min_lat': 35.99561061411391, 'max_lat': 36.01010126764238, 'min_long': -115.0794477471799, 'max_long': -115.06154773230813, 'category_counts': defaultdict(<class 'int'>, {'real estate agents': 3, 'home services': 3, 'real estate': 1, 'event planning & services': 1, 'golf': 1, 'restaurants': 1, 'home & rental insurance': 1})}
{'min_lat': 35.99561061411391, 'max_lat': 36.01010126764238, 'min_long': -115.06154773230813, 'max_long': -115.04364771743636, 'category_counts': defaultdict(<class 'int'>, {'golf': 1})}
{'min_lat': 35.99561061411391, 'max_lat': 36.01010126764238, 'min_long': -115.04364771743636, 'max_long': -115.02574770256459, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.99561061411391, 'max_lat': 36.01010126764238, 'min_long': -115.02574770256459, 'max_long': -115.00784768769282, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 36.01010126764238, 'max_lat': 36.0245919211708

...

{'min_lat': 36.25644237762648, 'max_lat': 36.270933031154954, 'min_long': -115.29424792564114, 'max_long': -115.27634791076937, 'category_counts': defaultdict(<class 'int'>, {'home services': 1, 'dog parks': 1, 'restaurants': 1, 'shutters': 1, 'active life': 1, 'gardeners': 1})}
{'min_lat': 36.25644237762648, 'max_lat': 36.270933031154954, 'min_long': -115.27634791076937, 'max_long': -115.2584478958976, 'category_counts': defaultdict(<class 'int'>, {'accessories': 1, 'auto repair': 2, 'food trucks': 1, 'car wash': 1, 'shopping': 4, 'general dentistry': 3, 'na': 1, 'real estate': 1, 'comic books': 1, 'home services': 1, 'pets': 1, 'shoe stores': 2, 'furniture stores': 1, 'nurseries & gardening': 1, 'fitness & instruction': 1, 'beauty & spas': 3, 'korean': 1, 'outdoor furniture stores': 1, 'restaurants': 5, 'oil change stations': 1, 'automotive': 2, 'pet services': 1, 'videos & video game rental': 1, 'orthodontists': 1, 'fashion': 3, 'dry cleaning & laundry': 1, 'middle schools & high sc

{'min_lat': 36.25644237762648, 'max_lat': 36.270933031154954, 'min_long': -115.09734776205167, 'max_long': -115.0794477471799, 'category_counts': defaultdict(<class 'int'>, {'home services': 1})}
{'min_lat': 36.25644237762648, 'max_lat': 36.270933031154954, 'min_long': -115.0794477471799, 'max_long': -115.06154773230813, 'category_counts': defaultdict(<class 'int'>, {'auto repair': 1})}
{'min_lat': 36.25644237762648, 'max_lat': 36.270933031154954, 'min_long': -115.06154773230813, 'max_long': -115.04364771743636, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 36.25644237762648, 'max_lat': 36.270933031154954, 'min_long': -115.04364771743636, 'max_long': -115.02574770256459, 'category_counts': defaultdict(<class 'int'>, {'party & event planning': 1})}
{'min_lat': 36.25644237762648, 'max_lat': 36.270933031154954, 'min_long': -115.02574770256459, 'max_long': -115.00784768769282, 'category_counts': defaultdict(<class 'int'>, {'no business fo

{'min_lat': 36.270933031154954, 'max_lat': 36.28542368468343, 'min_long': -115.18684783641052, 'max_long': -115.16894782153875, 'category_counts': defaultdict(<class 'int'>, {'beauty & spas': 3, 'banks & credit unions': 2, 'food': 3, 'drugstores': 2, 'chinese': 1, 'restaurants': 4, 'automotive': 2, 'pediatric dentists': 1, 'juice bars & smoothies': 1, 'donuts': 1, 'air duct cleaning': 1, 'business financing': 1, 'public services & government': 1, 'active life': 1, 'home services': 1, 'pet sitting': 1, 'beer': 1, 'tobacco shops': 1, 'local services': 1, 'shopping': 1, 'financial services': 1, 'american (traditional)': 1, 'health & medical': 2, 'pets': 1, 'convenience stores': 1, 'sushi bars': 1, 'dentists': 1, 'wedding planning': 1, 'hair removal': 1, 'child care & day care': 1, 'barbers': 1, 'dry cleaning & laundry': 1})}
{'min_lat': 36.270933031154954, 'max_lat': 36.28542368468343, 'min_long': -115.16894782153875, 'max_long': -115.15104780666698, 'category_counts': defaultdict(<class 

{'min_lat': 36.28542368468343, 'max_lat': 36.299914338211906, 'min_long': -115.24054788102583, 'max_long': -115.22264786615406, 'category_counts': defaultdict(<class 'int'>, {'home services': 1, 'gardeners': 1, 'trainers': 1, 'pool cleaners': 1})}
{'min_lat': 36.28542368468343, 'max_lat': 36.299914338211906, 'min_long': -115.22264786615406, 'max_long': -115.20474785128229, 'category_counts': defaultdict(<class 'int'>, {'public services & government': 1})}
{'min_lat': 36.28542368468343, 'max_lat': 36.299914338211906, 'min_long': -115.20474785128229, 'max_long': -115.18684783641052, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 36.28542368468343, 'max_lat': 36.299914338211906, 'min_long': -115.18684783641052, 'max_long': -115.16894782153875, 'category_counts': defaultdict(<class 'int'>, {'active life': 1, 'mexican': 1, 'it services & computer repair': 1, 'towing': 1, 'event planning & services': 1, 'health & medical': 4, 'pet stores': 1

{'min_lat': 36.299914338211906, 'max_lat': 36.31440499174038, 'min_long': -115.11524777692344, 'max_long': -115.09734776205167, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 36.299914338211906, 'max_lat': 36.31440499174038, 'min_long': -115.09734776205167, 'max_long': -115.0794477471799, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 36.299914338211906, 'max_lat': 36.31440499174038, 'min_long': -115.0794477471799, 'max_long': -115.06154773230813, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 36.299914338211906, 'max_lat': 36.31440499174038, 'min_long': -115.06154773230813, 'max_long': -115.04364771743636, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 36.299914338211906, 'max_lat': 36.31440499174038, 'min_long': -115.04364771743636, 'max_long': -115.02574770256459, 'category_counts':

### Write to file

In [12]:
with open('az_grid_boxes_category_counts.json', 'w') as outfile:
    json.dump(boxes, outfile)